<a href="https://colab.research.google.com/github/chitingchiang/style_transfer/blob/master/style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as pt
import matplotlib.style
matplotlib.style.use('default')
%matplotlib inline
import tensorflow as tf
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [4]:
model = VGG16(weights='imagenet', include_top=False)
model.count_params()

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


14714688

In [0]:
class BuildGraph:
    def __init__(n_pix=512, n_channel=3):
        self.content_img = tf.placeholder(dtype=tf.float32, shape=(1, n_pix, n_pix, n_channel))
        self.style_img = tf.placeholder(dtype=tf.float32, shape=(1, n_pix, n_pix, n_channel))
        self.combination_img = tf.Variable(tf.random.uniform(shape=(1, n_pix, n_pix, n_channel), minval=0, maxval=255)-127.5)
        
        self.content_weight = tf.placeholder(dtype=tf.float32, shape=())
        self.style_weight = tf.placeholder(dtype=tf.float32, shape=())
        self.variation_weight = tf.placeholder(dtype=tf.float32, shape=())
        
        self.lr_rate = tf.placeholder(dtype=tf.float32, shape=())
        
        tot_img = tf.concat([self.content_img, self.style_img, self.combination_img], axis=0)
        
        model = VGG16(weights='imagenet', include_top=False, input_tensor=tot_img)
        layers = dict([(layer.name, layer.output) for layer in model.layers])
        
        content_feature_layers = ['block2_conv2']
        content_loss = 0
        for layer_name in content_feature_layers:
            layer_features = layers[layer_name]
            content_loss += tf.reduce_sum(tf.square(layer_features[0, :, :, :]-layer_features[1, :, :, :]))
        content_loss = content_loss*self.content_weight
        
        style_feature_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3', 'block5_conv3']
        style_loss_constant = self.style_weight/((2*n_channel*n_pix**2)**2)/len(style_feature_layers)
        style_loss = 0
        for layer_name in style_feature_layers:
            layer_features = layers[layer_name]
            style_matrix = gram_matrix(layer_features[1, :, :, :])
            combination_matrix = gram_matrix(layer_features[2, :, :, :])
            style_loss = style_loss+tf.reduce_sum(tf.square(style_matrix-combination_matrix))
            
            return backend.sum(backend.square(S - C)) / (4. * (channels ** 2) * (size ** 2))
        style_loss = style_loss*style_loss_constant
        
        variation_loss = (tf.square(self.combination_img[:, :n_pix-1, :n_pix-1, :]
                                    -self.combination_img[:, 1:, :n_pix-1, :])+
                          tf.square(self.combination_img[:, :n_pix-1, :n_pix-1, :]
                                    -self.combination_img[:, :n_pix-1, 1:, :]))*self.variation_weight
        
        self.loss = content_loss+style_loss+variation_loss

        self.train_step = tf.train.AdamOptimizer(self.lr_rate).minimize(self.loss, var_list=[self.combination_img])
        
    def gram_matrix(x):
        x = tf.transpose(x, perm=(2, 0, 1))
        x = tf.reshape(x, (x.shape[0], -1))
        gram = tf.matmul(x, tf.transpose(x))
        return gram